# 03 - Train and Deploy

In [61]:
import boto3
import sagemaker
import time
from time import strftime

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")
region = boto_session.region_name
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

prefix = 'sagemaker-intel-dvc'

# print(f"account: {account}")
# print(f"bucket: {bucket}")
# print(f"region: {region}")
# print(f"role: {role}")

In [3]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import TensorBoardOutputConfig

In [4]:
dvc_repo_url = "codecommit::us-west-2://sagemaker-intel"
dvc_branch = "processed-dataset"

In [5]:
tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f's3://{bucket}/sagemaker-intel-logs',
    container_local_output_path='/opt/ml/output/tensorboard'
)

In [7]:
pt_estimator = PyTorch(
    base_job_name="training-intel-dataset",
    source_dir="scripts",
    entry_point="train.py",
    role=get_execution_role(),
    image_uri='****.dkr.ecr.us-west-2.amazonaws.com/emlo:train',
    instance_count=1,
    instance_type="ml.g4dn.2xlarge",
    tensorboard_output_config=tensorboard_output_config,
    use_spot_instances=True,
    max_wait=1800,
    max_run=1500,
    environment={
        "DVC_REPO_URL": dvc_repo_url,
        "DVC_BRANCH": dvc_branch,
        "GIT_USER": "m",
        "GIT_EMAIL": "m@emlo.com"
    }
)

In [ ]:
pt_estimator.fit()

## Deploy to Endpoint

In [6]:
from sagemaker.pytorch import PyTorchModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

Get the `model_data` from the models uploaded to S3 of the aboved training

Once your debugging is done, it's better to simply call `pt_estimator.deploy`

In [62]:
# pt_estimator.model_data

In [51]:
model = PyTorchModel(
    entry_point="infer.py",
    source_dir="script",
    role=get_execution_role(),
    model_data=pt_estimator.model_data,
    image_uri='***.dkr.ecr.us-west-2.amazonaws.com/emlo:infer',
)

In [52]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

--------!

## Inference

In [20]:
classnames = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [19]:
import torch

import torchvision.transforms as T
import torch.nn.functional as F

from PIL import Image
import numpy as np

In [59]:
import os
directory = 'img'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    inp_img1 = Image.open(f)
    input_array1 = np.array(inp_img1)/255
    input_tensor = {"inputs": input_array1}
    out = predictor.predict(input_tensor)
    print(f'the top 5 predictions for {f} (in %) are')
    print(out)
    print('------------------')
    

the top 5 predictions for img/street2.jpg (in %) are
{'street': 100.0, 'buildings': 0.0, 'sea': 0.0, 'glacier': 0.0, 'forest': 0.0}
------------------
the top 5 predictions for img/glacier2.jpg (in %) are
{'glacier': 99.65, 'mountain': 0.35, 'sea': 0.0, 'forest': 0.0, 'buildings': 0.0}
------------------
the top 5 predictions for img/forest2.jpg (in %) are
{'forest': 100.0, 'glacier': 0.0, 'sea': 0.0, 'street': 0.0, 'buildings': 0.0}
------------------
the top 5 predictions for img/mountain2.jpg (in %) are
{'mountain': 98.22, 'glacier': 1.09, 'buildings': 0.43, 'sea': 0.16, 'forest': 0.09}
------------------
the top 5 predictions for img/buildings2.jpg (in %) are
{'buildings': 58.88, 'sea': 22.97, 'mountain': 11.68, 'street': 3.55, 'glacier': 2.63}
------------------
the top 5 predictions for img/sea2.jpg (in %) are
{'sea': 99.88, 'glacier': 0.11, 'mountain': 0.0, 'street': 0.0, 'buildings': 0.0}
------------------
the top 5 predictions for img/buildings1.jpg (in %) are
{'buildings': 9

# Cleanup

In [60]:
predictor.delete_endpoint(delete_endpoint_config=True)
model.delete_model()